In [1]:
import quandl
quandl.ApiConfig.api_key = "KxKMd8dsHx1sWnaGj6_U"
import numpy as np
import matplotlib as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
def get_monthly_close_prices_ticker(ticker,start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SEP',ticker=ticker,date={'gte':start_date,'lte':end_date}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)
    close = df.pivot(index='date', columns='ticker', values='close')
    monthly = close.resample('M').apply(lambda x: x[-1])
    #.agg(lambda x: x[-1])
    return monthly

In [3]:
monthly_prices = get_monthly_close_prices_ticker('A','2009-13-31','2019-01-01')

In [4]:
#monthly_prices

In [5]:
#quandl.get_table('SHARADAR/SEP',date={'gte':'2018-06-30','lte':'2019-01-01'}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)

In [6]:
def get_monthly_returns(monthly_prices):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
        

    #.agg(lambda x: x[-1])
    monthly_returns = (monthly_prices - monthly_prices.shift(1))/monthly_prices.shift(1).fillna(value=0)
    
    
    return monthly_returns

In [7]:
df1 = get_monthly_returns(monthly_prices)


In [8]:
df1.head()

ticker,A
date,
2010-01-31,NaN
2010-02-28,0.122369
2010-03-31,0.093134
2010-04-30,0.054376
2010-05-31,-0.107557


In [9]:
df1 = df1.iloc[ 1: , : ]

In [10]:
df1.reset_index(inplace=True)


In [11]:
df1.head()

ticker,date,A
0,2010-02-28,0.122369
1,2010-03-31,0.093134
2,2010-04-30,0.054376
3,2010-05-31,-0.107557
4,2010-06-30,-0.121446


In [12]:
def get_factor_data_ticker(ticker,start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SF1',ticker=ticker,dimension='ARQ',calendardate={'gte':start_date, 'lte':end_date}, paginate=True).fillna(value=0)

    
    return df

In [13]:
df2 = get_factor_data_ticker('A','2009-13-31','2019-01-01')

In [14]:
df2.head()

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
0,A,ARQ,2018-12-31,2019-03-05,2019-01-31,2019-03-05,-381000000,8952000000,0,3712000000,...,317515869,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000
1,A,ARQ,2018-09-30,2018-12-20,2018-10-31,2019-03-05,-408000000,8541000000,0,3848000000,...,318533054,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000
2,A,ARQ,2018-06-30,2018-08-30,2018-07-31,2019-03-05,-335000000,8349000000,0,3667000000,...,318769547,320000000,324000000,3.759,4901000000,0,6000000,0,15.316,2653000000
3,A,ARQ,2018-03-31,2018-05-31,2018-04-30,2019-03-05,-310000000,8784000000,0,4525000000,...,319952126,322000000,326000000,3.745,5852000000,0,22000000,0,18.174,3160000000
4,A,ARQ,2017-12-31,2018-03-06,2018-01-31,2019-03-05,-269000000,8698000000,0,4397000000,...,322476579,323000000,323000000,3.749,5724000000,0,553000000,0,17.721,3036000000


In [15]:
df2['calendardate'] = pd.to_datetime(df2['calendardate']).dt.strftime("%Y-%m-%d")


In [16]:
def add_returns(row, df):
    c_date = row['calendardate']
    ticker = row['ticker']
    result = 0.0
    value = df.loc[df['date'] == c_date]
    if not value.empty:
        if ticker in value.columns:
            result = value[ticker].values[0]
    return result
df2['results'] = df2.apply(lambda row: add_returns(row, df1), axis=1)

In [17]:
df2.sort_values('calendardate')

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,results
None,,,,,,,,,,,,,,,,,,,,,
35,A,ARQ,2010-03-31,2010-06-07,2010-04-30,2019-03-05,-239000000,7767000000,0,5712000000,...,348000000,354000000,3.652,6997000000,0,31000000,0,20.106,3003000000,0.093134
34,A,ARQ,2010-06-30,2010-09-07,2010-07-31,2019-03-05,-225000000,9100000000,0,5735000000,...,347000000,352000000,3.988,7188000000,0,22000000,0,20.715,2818000000,-0.121446
33,A,ARQ,2010-09-30,2010-12-20,2010-10-31,2019-03-05,-88000000,9696000000,0,6169000000,...,344000000,356000000,4.581,7746000000,0,-49000000,0,22.517,3086000000,0.237301
32,A,ARQ,2010-12-31,2011-03-09,2011-01-31,2019-03-05,-63000000,8044000000,0,4598000000,...,347000000,355000000,4.378,6129000000,0,5000000,0,17.663,3192000000,0.183038
31,A,ARQ,2011-03-31,2011-06-07,2011-04-30,2019-03-05,278000000,8649000000,0,5096000000,...,347000000,355000000,4.833,6601000000,0,60000000,0,19.023,3504000000,0.064163
30,A,ARQ,2011-06-30,2011-09-07,2011-07-31,2019-03-05,271000000,8753000000,0,5223000000,...,348000000,357000000,4.859,6711000000,0,-49000000,0,19.284,3718000000,0.024865
29,A,ARQ,2011-09-30,2011-12-16,2011-10-31,2019-03-05,116000000,9057000000,0,5569000000,...,347000000,355000000,4.980,7061000000,0,4000000,0,20.349,3732000000,-0.152427
28,A,ARQ,2011-12-31,2012-03-05,2012-01-31,2019-03-05,87000000,9099000000,0,5715000000,...,348000000,352000000,4.698,7087000000,0,26000000,0,20.365,4010000000,-0.068533
27,A,ARQ,2012-03-31,2012-06-04,2012-04-30,2019-03-05,68000000,9413000000,0,6010000000,...,348000000,354000000,4.980,7422000000,0,38000000,0,21.328,4175000000,0.020403


In [18]:
corr_matrix = df2.corr().abs()

In [19]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [20]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [21]:
df2 = df2.drop(df2[to_drop], axis=1)

In [22]:
df2.shape

(36, 72)

In [23]:
dataset = df2.set_index('calendardate')




In [24]:
df_numbers = dataset.drop(columns=['ticker','dimension','reportperiod','lastupdated','datekey'])

In [25]:
#pd.options.display.max_columns = None
#display(df_numbers)

In [26]:
df_numbers.shape

(36, 66)

In [27]:
#df_numbers[:36]

In [28]:
dataset = df_numbers.values
dataset

array([[-3.81000000e+08,  8.95200000e+09,  0.00000000e+00, ...,
         0.00000000e+00,  2.61700000e+09, -6.75881133e-02],
       [-4.08000000e+08,  8.54100000e+09,  0.00000000e+00, ...,
         0.00000000e+00,  2.67700000e+09,  4.44181226e-02],
       [-3.35000000e+08,  8.34900000e+09,  0.00000000e+00, ...,
         0.00000000e+00,  2.65300000e+09, -1.29198966e-03],
       ...,
       [-8.80000000e+07,  9.69600000e+09,  0.00000000e+00, ...,
         0.00000000e+00,  3.08600000e+09,  2.37300704e-01],
       [-2.25000000e+08,  9.10000000e+09,  0.00000000e+00, ...,
         0.00000000e+00,  2.81800000e+09, -1.21446230e-01],
       [-2.39000000e+08,  7.76700000e+09,  0.00000000e+00, ...,
         0.00000000e+00,  3.00300000e+09,  9.31341386e-02]])

In [29]:
#dataset
36 * 0.9

32.4

In [30]:
train = dataset[0:32,:]
valid = dataset[32:,:]


In [31]:
train_data = dataset[:32]
test_data = dataset[32:]

train_targets = train_data[:,-1]

In [32]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_data = scaler.fit_transform(train_data)


In [33]:
test_data = scaler.fit_transform(test_data)

In [57]:
X_test = test_data

In [34]:
train_targets = train_targets.reshape(-1, 1)
train_targets = scaler.fit_transform(train_targets)

In [47]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [54]:
x_train = np.reshape(train_data, (train_data.shape[0],train_data.shape[1],1))
y_train = train_targets

In [58]:
X_test = np.reshape(train_data, (train_data.shape[0],train_data.shape[1],1))

In [55]:
x_train.shape

(32, 66, 1)

In [56]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)


Instructions for updating:
Use tf.cast instead.
Epoch 1/1
 - 3s - loss: 0.1241


In [59]:
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [68]:
X_test.shape

(32, 66, 1)

In [61]:
closing_price

array([[-0.01883256],
       [-0.01451773],
       [-0.02202526],
       [-0.01609451],
       [ 0.02140378],
       [-0.01338538],
       [-0.0158553 ],
       [-0.01755404],
       [-0.01826574],
       [-0.01654363],
       [-0.01990165],
       [-0.01153719],
       [-0.01532416],
       [-0.01924205],
       [-0.02658634],
       [-0.01720296],
       [-0.01008006],
       [ 0.00921216],
       [ 0.00388581],
       [ 0.01295887],
       [ 0.0184966 ],
       [ 0.01441005],
       [-0.00729515],
       [ 0.00420687],
       [ 0.01450097],
       [ 0.00068078],
       [-0.00863087],
       [ 0.01441225],
       [ 0.01988488],
       [ 0.00620365],
       [ 0.0080451 ],
       [ 0.02130242]], dtype=float32)

In [66]:
dataset[32:,-1]

array([ 0.18303826,  0.2373007 , -0.12144623,  0.09313414])

In [64]:
rms=np.sqrt(np.mean(np.power((valid[-1]-closing_price),2)))
rms

2020553174.3090613

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model


In [ ]:
import numpy as np

k = 4
num_val_samples = len(train) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    val_mse = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mse)



In [ ]:
all_scores

In [ ]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)


In [ ]:
x_train = np.reshape(x_train.shape[0],x_train.shape[1],1) 

In [ ]:
x_train = np.reshape(x_train.shape[0],x_train.shape[1],1) 

In [ ]:
x_train.shape[0]

In [ ]:
x_train.shape[1]

In [ ]:
from sklearn.preprocessing import scale

# Remove unimportant features (weekdays)
train_features = train_features.iloc[:, :-4]
test_features = test_features.iloc[:, :-4]

# Standardize the train and test features
scaled_train_features = scale(train_features)
scaled_test_features = scale(test_features)

# Plot histograms of the 14-day SMA RSI before and after scaling
f, ax = plt.subplots(nrows=2, ncols=1)
train_features.iloc[:, 2].hist(ax=ax[0])
ax[1].hist(scaled_train_features[:, 2])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model_1 = Sequential()
model_1.add(Dense(100, input_dim=scaled_train_features.shape[1], activation='relu'))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dense(1, activation='linear'))

# Fit the model
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(scaled_train_features, train_targets, epochs=25)